# Inside the Leak

As global markets expand and become more interconnected, businesses are increasingly looking for resources to help identify competitive and profitable opportunities. Several data leakages in the last years have shown that a common approach to this is the creation of offshore companies, i.e. companies created in low-tax, offshore jurisdictions.
Our goal is to analyze motivating factors for creating
such entities. We believe that a better understanding of the reasons can help to find ways to deal with those tendencies. This has an impact on the social good, because fiscal prudence and opennesss in international trade can have a powerful effect on improving society.

Our analysis is based on data provided in the [Offshore Leaks Database](https://www.occrp.org/en/panamapapers/database). It contains information about 500,000 offshore companies, foundations and trusts including links to people and companies in more than 200 countries and territories. The information comes from the Panama Papers, the Offshore Leaks and the Bahamas Leaks investigations cunducted in the years 2013 to 2016. While those data leaks contain diverse sorts of documents from emails to bank documents, the database provides structured information excluding raw files. The latest investigation on the Paradise Papers that was released in November 2017 is not included in our analysis.

In order to better understand the underlying structures of the offshore business we analyze the available data on the level of countries. We identify the most involved countries and try to find factors that characterize those countries. To this aim we enrich the dataset with information about the economical and social background of countries from the [Index of Economic Freedom](http://www.heritage.org/index/about).
Furthermore, we investigate how the different countries are connected and how their presence in the offshore business evolved in the last years. In particular, we wanted to see whether the publication of the leaks influenced those developments in some way.

In [1]:
#TODO explain terminology + add things if needed

Before we present our findings we want to clarify some of the used terminonology:
    -  offshore entity (explain that not per se illegal?)
    -  jurisdiction
    -  incoorporation

## Most involved countries and their characteristics

In this section we analyze which countries are most involved in the offshore business and try to identify their economic characteristics.

First of all, we want to give an overview of the available data.

In [2]:
# TODO:
# - explain missing data for tax havens
# - most involved jurisdictions
# - most involved tax havens
# - correlation nrofoffshores + index data + CONCLUSION

## International relations

In this section we present our findings about how the different countries are connected.

In [3]:
# TODO: shortly explain clustering
#present + interpret matrices
#present + interpret map
#present + interpret PCA

## Development of the countries presence

In [4]:
# see readme :-)